<a href="https://colab.research.google.com/github/xCosmicx/ATA/blob/main/week12/transformer_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Quick Tour of What Transformers can do?

In this exercise, we will use Hugging Face 🤗 Transformer library to perform some common NLP tasks. 

We will use the pipeline() function, which supports several NLP tasks such as classification, summarization, machine translation and so on. For a list of support tasks see the documentation [here](https://huggingface.co/transformers/main_classes/pipelines.html#transformers.pipeline). Pipeline connects a task-specific model with its necessary preprocessing and postprocessing steps, allowing us to directly input any text and get an intelligible answer. 

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
# install the extra package "sentencepiece" required for machine translation tasks
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 15.0 MB/s 
     |████████████████████████████████| 4.4 MB 64.1 MB/s 
     |████████████████████████████████| 1.1 MB 61.4 MB/s 
     |████████████████████████████████| 140 kB 70.5 MB/s 
     |████████████████████████████████| 101 kB 13.0 MB/s 
     |████████████████████████████████| 212 kB 74.8 MB/s 
     |████████████████████████████████| 596 kB 76.9 MB/s 
     |████████████████████████████████| 127 kB 78.7 MB/s 
     |████████████████████████████████| 271 kB 77.6 MB/s 
     |████████████████████████████████| 144 kB 73.4 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
     |████████████████████████████████| 6.6 MB 53.9 MB/s 
     |████████████████████████████████| 1.2 MB 60.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urlli

### Sentiment Analysis 

This pipeline uses the default model fine-tuned on Stanford Sentiment Treebank v2 dataset - "distilbert-base-uncased-finetuned-sst-2-english" to classify if a text express positive or negative sentiment. 

If you want to use other models available from Hugging Face models library, you can specify it in the parameter `pipeline("sentiment-analysis", model=???)`.  


In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I fell asleep halfway through the show.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

[{'label': 'NEGATIVE', 'score': 0.9996592998504639}]

### Zero-shot classification 

Zero-shot-classification pipeline allows you to specify which labels to use for the classification, so you don’t have to rely on the labels of the pretrained model. You can classify the text using the labels you specified. 

This is especially useful for real-world projects where you have a lot of unlabelled data and do not have the resources to annotate your data. Zero-shot classification allows you to quickly annotate your dataset.

In [3]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "The CO2 emission has been growing at an alarming rate.",
    candidate_labels=["environment", "politics", "business"],
)

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

{'labels': ['environment', 'business', 'politics'],
 'scores': [0.8911433815956116, 0.08387470990419388, 0.024981897324323654],
 'sequence': 'The CO2 emission has been growing at an alarming rate.'}

### Text Generation

Now let’s see how to use a pipeline to generate some text. The main idea here is that you provide a prompt and the model will auto-complete it by generating the remaining text. This is similar to the predictive text feature that is found on many phones. Text generation involves randomness, so it’s normal if you don’t get the same results as shown below. The default model used is gpt-2.

You can control how many different sequences are generated with the argument `num_return_sequences` and the total length of the output text with the argument `max_length`.



In [4]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("Harry Potter whipped out his wand and", max_length=50, num_return_sequences=2)

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Harry Potter whipped out his wand and turned it out of the bag, turning it into a red light. The wand was red in colour and only black at the top. The wand that Ron stole came from a dead end shop in London, and in'},
 {'generated_text': 'Harry Potter whipped out his wand and was about to do with the Dark Lord with a spell, when Harry realized this, he took a sharp breath and turned his wand around.\n\n"Harry Potter! Why are you here?!"\n\nThe old'}]

### Mask filling

The next pipeline you’ll try is fill-mask. The idea of this task is to fill in the blanks in a given text. 

The top_k argument controls how many possibilities you want to be displayed. Note that here the model fills in the special <mask> word, which is often referred to as a mask token. Other mask-filling models might have different mask tokens, so it’s always good to verify the proper mask word when exploring other models. 

In [5]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("The tech giant has been accused of trademark <mask> by other companies.", top_k=2)

No model was supplied, defaulted to distilroberta-base (https://huggingface.co/distilroberta-base)


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

[{'score': 0.8685477375984192,
  'sequence': 'The tech giant has been accused of trademark infringement by other companies.',
  'token': 26461,
  'token_str': ' infringement'},
 {'score': 0.04818866029381752,
  'sequence': 'The tech giant has been accused of trademark theft by other companies.',
  'token': 5751,
  'token_str': ' theft'}]

### Named Entity Recognition

Named entity recognition (NER) is a task where the model has to find which parts of the input text correspond to entities such as persons, locations, or organizations. For example, `Michael Bloomberg` was the ex-mayor of `New York`. Michael Bloomberg will be identified as PER, whereas New York will be identified as LOC. 

We pass the option `grouped_entities=True` in the pipeline creation function to tell the pipeline to regroup together the parts of the sentence that correspond to the same entity, e.g. "Michael" and "Bloomberg" are parts that refer to the same person.

In [6]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("Lee Kuan Yew previously lived at 38 Oxley Road.")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english)


Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:136: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  "`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to"


[{'end': 12,
  'entity_group': 'PER',
  'score': 0.93581307,
  'start': 0,
  'word': 'Lee Kuan Yew'},
 {'end': 46,
  'entity_group': 'LOC',
  'score': 0.89072376,
  'start': 36,
  'word': 'Oxley Road'}]

### Question Answering

The question-answering pipeline answers questions using information from a given context. Note that the answer is extracted from the given context and not generated. The `start` and `end` in the example below tells you the span of the text in the context that provide the answer.

In [7]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="What am I specializing in?",
    context="I am currently taking Diploma in Information Technology in NYP, with specialization in Artificial Intelligence.",
)

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

{'answer': 'Artificial Intelligence',
 'end': 110,
 'score': 0.3812722861766815,
 'start': 87}

### Summarization

Summarization is the task of reducing a text into a shorter text while keeping all (or most) of the important aspects referenced in the text. Like with text generation, you can specify a `max_length` or a `min_length` for the result.

In [8]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

[{'summary_text': ' America has changed dramatically during recent years . The number of engineering graduates in the U.S. has declined in traditional engineering disciplines such as mechanical, civil,    electrical, chemical, and aeronautical engineering . Rapidly developing economies such as China and India continue to encourage and advance the teaching of engineering .'}]

### Translation

You can choose the model that corresponds to the language pair you want to translate. For example, if you want to translate from fr to en, you need to choose the model that has a naming like "mt-fr-en".  Similary, if I want to translate from English to Chinese, then it should be "mt-en-zh".

In [9]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh")
translator("Singapore government is doing a good job in tackling pandemic.")

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/788k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': '新加坡政府在防治流行病方面正在做很好的工作。'}]